#### KNN - Visão geral

Antes de continuar no livro*, decidi estudar knn por fora para ter uma visão básica do tema.

Aula disponível em: https://www.youtube.com/watch?v=xL2RK0QYtHc

* Ganesh, Tinniam V. Practical Machine Learning with R and Python: Machine Learning in Stereo (p. 99). Edição do Kindle. 

#### 0. Importando as bibliotecas

In [3]:
# Manipular/ver dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Criar modelo ML
from sklearn import preprocessing, model_selection, neighbors
from sklearn.preprocessing import StandardScaler

# Plotar o modelo
from mlxtend.plotting import plot_decision_regions